In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [2]:
#url used 
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')  # forming a soup object
#print(soup.prettify())
#my_table = soup.find('table',{'class':'wikitable sortable'})
#print(my_table)
Postal_Code,Borough,Neighbourhood=[],[],[]      # empty lists to store row values
for items in soup.find('table', class_='wikitable sortable').find_all('tr'):        #in table field find all rows
    data = items.find_all(['td'])   #can ignore the header field
    #print(data)
    try:
        Postal_Code.append(data[0].find(text=True).replace('\n',''))   #replace to remove newline character
        Borough.append(data[1].find(text=True).replace('\n',''))
        Neighbourhood.append(data[2].find(text=True).replace('\n',''))
        #print(Postal_Code,Borough,Neighbourhood)
    except:
        pass
    
    
#Form a Dataframe 
df=pd.DataFrame(Postal_Code,columns=['Postal_Code'])
df['Borough']=Borough
df['Neighbourhood']=Neighbourhood
df_n=df[df['Borough']!="Not assigned"]    # new dataframe after dropping rows with not assigned value in 'Neighbourhood column'
df_n.reset_index(drop=True,inplace=True)
#df_n[df_n.Neighbourhood == "Not assigned"]

In [3]:
#To assign the Borough field to Neighbourhood in case it is Not assigned
for i in range(len(df_n)):
    if df_n['Neighbourhood'].iloc[i]=="Not assigned":
        df_n['Neighbourhood'].iloc[i]=df_n['Borough'].iloc[i]
        print(df_n['Neighbourhood'])

In [4]:
df_n = df_n.groupby(['Postal_Code', 'Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).to_frame()
df_n.reset_index(inplace=True)

In [5]:
#Display the shape of final dataframe
print(df_n.shape)

(103, 3)
